In [1]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Input, Activation, AveragePooling2D, MaxPool2D, concatenate, GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.initializers import VarianceScaling
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model, load_model

In [7]:
#### DEFINING THE CONVOLUTIONS

def convolution(inputs, num_filters, kernel_size, padding, strides):

    X = Conv2D(num_filters, kernel_size=kernel_size, padding=padding, strides=strides, kernel_initializer=VarianceScaling(scale=2.0, distribution='normal'), kernel_regularizer=l2(0.0001))(inputs)
    X = BatchNormalization(axis=-1, momentum=0.99, scale=False)(X)
    X = Activation('relu')(X)

    return X

In [4]:
#### DEFINING INCEPTION BLOCK A

def inception_block_a(inputs):

    branch_1 = AveragePooling2D(pool_size=(3,3), strides=(1,1), padding='same')(inputs)
    branch_1 = convolution(branch_1, 96, (1,1), padding='same', strides=(1,1))

    branch_2 = convolution(inputs, 96, (1,1), padding='same', strides=(1,1))

    branch_3 = convolution(inputs, 64, (1,1), padding='same', strides=(1,1))
    branch_3 = convolution(branch_3, 96, (3,3), padding='same', strides=(1,1))

    branch_4 = convolution(inputs, 64, (1,1), padding='same', strides=(1,1))
    branch_4 = convolution(branch_4, 96, (3,3), padding='same', strides=(1,1))
    branch_4 = convolution(branch_4, 96, (3,3), padding='same', strides=(1,1))

    output = concatenate([branch_1, branch_2, branch_3, branch_4], axis=-1)

    return output

In [5]:
#### DEFINING THE INCEPTION BLOCK B

def inception_block_b(inputs):

    branch_1 = AveragePooling2D(pool_size=(3,3), strides=(1,1), padding='same')(inputs)
    branch_1 = convolution(branch_1, 128, (1,1), strides=(1,1), padding='same')

    branch_2 = convolution(inputs, 384, (1,1), padding='same', strides=(1,1))

    branch_3 = convolution(inputs, 192, (1,1), padding='same', strides=(1,1))
    branch_3 = convolution(branch_3, 224, (1,7), padding='same', strides=(1,1))
    branch_3 = convolution(branch_3, 256, (1,7), padding='same', strides=(1,1))

    branch_4 = convolution(inputs, 192, (1,1), padding='same', strides=(1,1))
    branch_4 = convolution(branch_4, 192, (1,7), padding='same', strides=(1,1))
    branch_4 = convolution(branch_4, 224, (7,1), padding='same', strides=(1,1))
    branch_4 = convolution(branch_4, 224, (1,7), padding='same', strides=(1,1))
    branch_4 = convolution(branch_4, 256, (7,1), padding='same', strides=(1,1))

    output = concatenate([branch_1, branch_2, branch_3, branch_4], axis=-1)

    return output

In [6]:
#### DEFINING THE INCEPTION BLOCK C

def inception_block_c(inputs):

    branch_1 = AveragePooling2D(pool_size=(3,3), strides=(1,1), padding='same')(inputs)
    branch_1 = convolution(branch_1, 256, (1,1), padding='same', strides=(1,1))

    branch_2 = convolution(inputs, 256, (1,1), padding='same', strides=(1,1))
  
    branch_3 = convolution(inputs, 384, (1,1), padding='same', strides=(1,1))
    branch_3_1 = convolution(branch_3, 256, (1,3), padding='same', strides=(1,1))
    branch_3_2 = convolution(branch_3, 256, (3,1), padding='same', strides=(1,1))

    branch_4 = convolution(inputs, 384, (1,1), padding='same', strides=(1,1))
    branch_4 = convolution(branch_4, 448, (1,3), padding='same', strides=(1,1))
    branch_4 = convolution(branch_4, 512, (3,1), padding='same', strides=(1,1))
    branch_4_1 = convolution(branch_4, 256, (3,1), padding='same', strides=(1,1))
    branch_4_2 = convolution(branch_4, 256, (1,3), padding='same', strides=(1,1))

    output = concatenate([branch_1, branch_2, branch_3_1, branch_3_2, branch_4_1, branch_4_2], axis=-1)

    return output

In [8]:
#### DEFINING THE RESIDUAL BLOCK A

def residual_block_a(inputs):

    branch_1 = MaxPool2D(pool_size=(3,3), strides=(2,2), padding='valid')(inputs)

    branch_2 = convolution(inputs, 384, (3,3), padding='valid', strides=(2,2))

    branch_3 = convolution(inputs, 192, (1,1), padding='same', strides=(1,1))
    branch_3 = convolution(branch_3, 224, (3,3), padding='same', strides=(1,1))
    branch_3 = convolution(branch_3, 256, (3,3), padding='valid', strides=(2,2))

    output = concatenate([branch_1, branch_2, branch_3], axis=-1)

    return output

In [9]:
#### DEFINING THE RESIDUAL BLOCK B

def residual_block_b(inputs):

    branch_1 = MaxPool2D(pool_size=(3,3), strides=(2,2), padding='valid')(inputs)

    branch_2 = convolution(inputs, 192, (1,1), padding='same', strides=(1,1))
    branch_2 = convolution(branch_2, 192, (3,3), padding='valid', strides=(2,2))

    branch_3 = convolution(inputs, 256, (1,1), padding='same', strides=(1,1))
    branch_3 = convolution(branch_3, 256, (1,7), padding='same', strides=(1,1))
    branch_3 = convolution(branch_3, 320, (7,1), padding='same', strides=(1,1))
    branch_3 = convolution(branch_3, 320, (3,3), padding='valid', strides=(2,2))

    output = concatenate([branch_1, branch_2, branch_3], axis=-1)

    return output

In [10]:
#### DEFINING THE STEM

def stem(inputs):

    layer_1 = convolution(inputs, 32, (3,3), padding='valid', strides=(2,2))
    layer_2 = convolution(layer_1, 32, (3,3), padding='valid', strides=(1,1))
    layer_3 = convolution(layer_2, 64, (3,3), padding='same', strides=(1,1))
    layer_3_1 = MaxPool2D(pool_size=(3,3), strides=(2,2), padding='valid')(layer_3)
    layer_3_2 = convolution(layer_3, 96, (3,3), padding='valid', strides=(2,2))
    concat_1 = concatenate([layer_3_1, layer_3_2], axis=-1)
    concat_branch_1 = convolution(concat_1, 64, (1,1), padding='same', strides=(1,1))
    concat_branch_1 = convolution(concat_branch_1, 96, (3,3), padding='valid', strides=(1,1))
    concat_branch_2 = convolution(concat_1, 64, (1,1), padding='same', strides=(1,1))
    concat_branch_2 = convolution(concat_branch_2, 64, (7,1), padding='same', strides=(1,1))
    concat_branch_2 = convolution(concat_branch_2, 64, (1,7), padding='same', strides=(1,1))
    concat_branch_2 = convolution(concat_branch_2, 96, (3,3), padding='valid', strides=(1,1))
    concat_2 = concatenate([concat_branch_1, concat_branch_2], axis=-1)
    concat_2_branch_1 = convolution(concat_2, 192, (3,3), padding='valid', strides=(2,2))
    concat_2_branch_2 = MaxPool2D(pool_size=(3,3), strides=(2,2), padding='valid')(concat_2)

    output = concatenate([concat_2_branch_1, concat_2_branch_2], axis=-1)

    return output

In [16]:
#### DEFINING THE INCEPTION V4 NETWORK

def inceptionv4(input_shape, classes):

    inputs = Input(input_shape)

    layer_1 = stem(inputs)

    layer_2 = inception_block_a(layer_1)
    layer_2 = inception_block_a(layer_2)
    layer_2 = inception_block_a(layer_2)
    layer_2 = inception_block_a(layer_2)

    layer_3 = residual_block_a(layer_2)

    layer_4 = inception_block_b(layer_3)
    layer_4 = inception_block_b(layer_4)
    layer_4 = inception_block_b(layer_4)
    layer_4 = inception_block_b(layer_4)
    layer_4 = inception_block_b(layer_4)
    layer_4 = inception_block_b(layer_4)
    laeyr_4 = inception_block_b(layer_4)

    layer_5 = residual_block_b(layer_4)

    layer_6 = inception_block_c(layer_5)
    layer_6 = inception_block_c(layer_6)
    layer_6 = inception_block_c(layer_6)

    layer_7 = GlobalAveragePooling2D()(layer_6)

    layer_8 = Dropout(0.2)(layer_7)

    layer_9 = Dense(classes, activation='softmax')(layer_8)

    model = Model(inputs=inputs, outputs=layer_9)

    return model

In [14]:
model = inceptionv4(input_shape=(299, 299, 3), classes=6)

In [15]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_149 (Conv2D)             (None, 149, 149, 32) 896         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_149 (BatchN (None, 149, 149, 32) 96          conv2d_149[0][0]                 
__________________________________________________________________________________________________
activation_149 (Activation)     (None, 149, 149, 32) 0           batch_normalization_149[0][0]    
____________________________________________________________________________________________